# Problem 1 - Find the first and latest transactions based on purchase_time of each buyer_idin each store_id and sort by buyer_id and store_id.
```
SELECT *
FROM (
  SELECT *, MAX(rank) OVER (PARTITION BY buyer_id, store_id) AS last
  FROM (
   SELECT *,
    ROW_NUMBER() OVER (PARTITION BY buyer_id, store_id
                      ORDER BY purchase_time) AS rank
   FROM `jason-366210.ftds_p1_w4_d3_am.purchase`
  )
)
WHERE rank=1 OR rank=last
ORDER BY buyer_id, store_id;
```

option 2

```
SELECT buyer_id, store_id,
 MIN(purchase_time) AS start_date,
 MAX(purchase_time) AS end_date
FROM `jason-366210.ftds_p1_w4_d3_am.purchase`
GROUP BY buyer_id, store_id;
```

# Problem 2 - Find buyers that make more than 1 order in 1 month. Show buyer_id, year  of  purchase,  month  of  purchase, and total order in that month.

```
SELECT buyer_id, year, month, count
FROM (
  SELECT *, MAX(count) OVER (PARTITION BY buyer_id, month) AS last
  FROM (
   SELECT *,
    ROW_NUMBER() OVER (PARTITION BY buyer_id, month) AS count
   FROM (
    SELECT *, EXTRACT(YEAR FROM purchase_time) AS year, 
     EXTRACT(MONTH FROM purchase_time) AS month
    FROM `jason-366210.ftds_p1_w4_d3_am.purchase`
   )
  )
)
WHERE count > 1 AND count=last
ORDER BY buyer_id, year, month;
```

option 2

```
SELECT buyer_id, year, month,
 COUNT(*) AS total_order
FROM (
  SELECT *,
   EXTRACT(YEAR FROM purchase_time) AS year,
   EXTRACT(MONTH FROM purchase_time) AS month
   FROM `jason-366210.ftds_p1_w4_d3_am.purchase`
)
GROUP BY buyer_id, year, month
 HAVING total_order > 1;
```

# Problem 3 - Get all transactions with purchase_price lower than average purchase_price in each buyer_location. Show buyer_location, product_id, category_name, purchase_price,  and average purchase_price
```
SELECT *
FROM (
  SELECT a.buyer_location, b.product_id, c.category_name, 
   b.purchase_price,
   AVG(purchase_price) 
    OVER (PARTITION BY a.buyer_location) AS avg_purchase_price
  FROM `jason-366210.ftds_p1_w4_d3_am.buyer` AS a
  JOIN `jason-366210.ftds_p1_w4_d3_am.purchase` AS b
   ON a.buyer_id=b.buyer_id
  JOIN `jason-366210.ftds_p1_w4_d3_am.product` AS c
   ON b.product_id=c.product_id
)
WHERE purchase_price < avg_purchase_price;
```

# Problem 4 - Get all transactions with purchase_price lower than average purchase_price of products within the corresponding category. Show category_name, product_id, purchase_id, purchase_price,  and  average purchase_price  in  corresponding  category  with  sorted  by purchase_id andproduct_id.
```
SELECT *
FROM (
  SELECT c.category_name, b.product_id, b.purchase_id, b.purchase_price,
   AVG(purchase_price) 
    OVER (PARTITION BY c.category_name) AS avg_purchase_price
  FROM `jason-366210.ftds_p1_w4_d3_am.purchase` AS b
  JOIN `jason-366210.ftds_p1_w4_d3_am.product` AS c
   ON b.product_id=c.product_id
)
WHERE purchase_price < avg_purchase_price
ORDER BY purchase_id, product_id;
```

# Problem 5 - Get top 3 products from successful transactions with the highest purchase_price in October 2021 in each category. Show the category_name, product_id, purchase_time, and purchase_price.

```
SELECT *
FROM (
  SELECT *,
   ROW_NUMBER() OVER (PARTITION BY year, month, category_name 
                      ORDER BY purchase_time) AS rank
  FROM (
   SELECT c.category_name, b.product_id, b.purchase_time, 
    b.purchase_price,
    EXTRACT(YEAR FROM b.purchase_time) AS year, 
    EXTRACT(MONTH FROM b.purchase_time) AS month,
   FROM `jason-366210.ftds_p1_w4_d3_am.purchase` AS b
   JOIN `jason-366210.ftds_p1_w4_d3_am.product` AS c
    ON b.product_id=c.product_id
   WHERE b.payment_status='paid'
  )
)
WHERE rank <= 3 AND year=2021 AND month=10;
```

# Problem 6 - Find the top  10  buyers  on  a  per-purchase  basis with the highest price from location Jakarta & Malang. Hint  : To  find  these  buyers,  you  don't  need  to  sum  up  their  transactions  per  each  buyer. Based on the question, you just find transactions with the highest purchase_price.
```
SELECT *
FROM (
  SELECT a.buyer_id, a.buyer_location, b.purchase_price,
   ROW_NUMBER() OVER (PARTITION BY a.buyer_location 
                      ORDER BY purchase_price DESC) AS rank
  FROM `jason-366210.ftds_p1_w4_d3_am.buyer` AS a
  JOIN `jason-366210.ftds_p1_w4_d3_am.purchase` AS b
   ON a.buyer_id=b.buyer_id
  WHERE a.buyer_location IN ("Jakarta", "Malang")
)
WHERE rank <= 10;
```

# Problem 7 - Get  top  3 store_id  with  the  highest purchase_price  from  a purchase/transaction in each buyer_location. Show the store_id, buyer_location, purchase_price.(Hint : only select transactions with payment_status is paid)
```
SELECT *
FROM (
  SELECT b.store_id, a.buyer_location, b.purchase_price,
   ROW_NUMBER() OVER (PARTITION BY a.buyer_location
                      ORDER BY b.purchase_price DESC) AS rank
  FROM `jason-366210.ftds_p1_w4_d3_am.buyer` AS a
  JOIN `jason-366210.ftds_p1_w4_d3_am.purchase`AS b
   ON a.buyer_id=b.buyer_id
  WHERE b.payment_status='paid'
)
WHERE rank <= 3
ORDER BY buyer_location, rank;
```